In [1]:
import torch
import torch.nn as nn
import numpy as np

import os
os.environ['CUDA_VISIBLE_DEVICES'] = "1, 2, 3"

In [3]:
class Arguments:
    res_path = './results'
    model_path = './models'
    epoch  = 20
    patience = -1
    teacher = False
    bidi = False
    test = False
    shrd_dec_emb = False 
    btstrp  = None 
    lm = False 
    toy = False 
    pretty = False 
    mmi = False
    seq2seq = False 
    drp = 0.3
    num_lyr = 1
    lr = 0.0001
    bt_siz = 138
    beam = 1
    vocab_size = 50005 
    response_vocab_size = 32149
    emb_size = 300
    hid_size = 256
    ut_hid_size = 256
    ses_hid_size = 512
    dec_hid_size = 512
    kb_embed_size = 100
    kb_size = 9274341
    kb_relations_size= 571
    use_embed = True




In [4]:
options = Arguments()

In [5]:

ent_embed = np.load('transe_dir/ent_embed.pkl.npy')
rel_embed = np.load('transe_dir/rel_embed.pkl.npy')

new_row = np.zeros((1,100), dtype=np.float32)
    
ent_embed = np.vstack([new_row, ent_embed]) # corr. to <pad_kb>
ent_embed = np.vstack([new_row, ent_embed]) # corr. to <nkb>

rel_embed = np.vstack([new_row, rel_embed]) # corr. to <pad_kb>
rel_embed = np.vstack([new_row, rel_embed]) 



In [6]:
def mini_batch_processing(mini_batch):
    mem_size = len(mini_batch[0][6].split('|'))
    if mem_size > 10000 :
        mem_size =  10000
    u1_text = np.array([s[0][0] for s in mini_batch])
    u1_kb   = np.array([s[1][0] for s in mini_batch])
    u2_text = np.array([s[0][1] for s in mini_batch] )
    u2_kb   = np.array([s[1][1] for s in mini_batch])
    target_kb = np.array([s[2] for s in mini_batch])
    target_utterance = np.array([s[3] for s in mini_batch])
    cand_ent =  [ [int(i) for i in s[6].split('|')] for s in mini_batch]
    cand_ent =  [ s[:-1][:mem_size-1]+[1]  for s in cand_ent]
    cand_ent =  np.array([ s + [0]*(mem_size-len(s)) for s in cand_ent])
    
    cand_rel =  [ [int(i) for i in s[7].split('|')] for s in mini_batch]
    cand_rel =  [ s[:-1][:mem_size-1]+[1]  for s in cand_rel]
    cand_rel =  np.array([ s + [0]*(mem_size - len(s)) for s in cand_rel])
    
    cand_val =  [ [int(i) for i in s[8].split('|')] for s in mini_batch]
    cand_val =  [ s[:-1][:mem_size-1]+[1]  for s in cand_val]
    cand_val =  np.array([s + [0]*(mem_size-len(s)) for s in cand_val])
    
    mem_weights = ((cand_val!=0) * (cand_val!=1) ).astype(int)
    '''
   
    labels = []
    candidates = []
    for i in range(64):
        positive_samples_i =list( set(target_kb[i])-{0})
        labels_i = [1]*len(positive_samples_i)
        negative_samples_i = list(set(cand_val[i])-(set(target_kb[i])-{0}) )
        if len(negative_samples_i) > 0:
            negative_samples_subset_i  = list( np.random.choice(negative_samples_i,20 - len(labels_i)) )
        else :
            negative_samples_subset_i =[]

        labels_i = labels_i + [0] *(20 - len(labels_i))
        candidates_i = positive_samples_i + negative_samples_subset_i
        candidates_i = candidates_i + (20-len(candidates_i)) *[0]

        labels.append(labels_i)
        candidates.append( candidates_i )
    
    c = np.array(labels)
    candidates_pred = np.array(candidates)
    
    '''
    
    b , l = cand_val.shape
    c = np.zeros((b, l))
    for i in range(b):
        for j in range(l):
            c[i,j] = int( (cand_val[i,j] in target_kb[i]) and cand_val[i,j] != 0) 
    
    
    
    return(u1_text, u1_kb, u2_text, u2_kb, target_kb, target_utterance,  cand_ent, cand_rel, cand_val,c, mem_weights)


In [7]:
loss_dec =  nn.CrossEntropyLoss(size_average=False, ignore_index=3)
softmax = nn.Softmax(dim = 1)

In [8]:
from model import  *

In [9]:
import os

In [10]:
def padded_candidates(t):
    n_samples = len(t)
    
    for i in range(n_samples):
        cand = t[i][6].split('|')[:-1]
        n_cand = len(cand)
        if n_cand  < 10:
            t[i][6] = '|'.join( cand + ['0']*(9-n_cand)) +'|1'
            cand = t[i][7].split('|')[:-1]
            t[i][7] = '|'.join( cand + ['0']*(9-n_cand)) + '|1'
            cand = t[i][8].split('|')[:-1]
            t[i][8] = '|'.join( cand + ['0']*(9-n_cand)) + '|1'
    return(t)

In [11]:
training_files=["dump/"+x for x in os.listdir("/home/tony/QA/dump") if x.startswith('train_')]

training_files = training_files+ ["dump/"+x for x in os.listdir("/home/tony/QA/dump") \
                if x.startswith('valid_')]



In [12]:
testing_file ='dump/test_preprocessed/test_data_file_verify.pkl'

In [13]:
training_files =   training_files + [testing_file]

In [14]:
import pickle as pkl

In [25]:
model_ = model(options)

model_.load_state_dict(torch.load('train.pt'))

In [26]:
model_ = nn.DataParallel(model_).cuda()

In [27]:
from torch import optim
from tqdm import tqdm_notebook

In [28]:
param_kg_loss = []
param_dec_loss = []
param_kg = []
param_dec = []
for t in model_.named_parameters():
    if 'kg_predict' in t[0]  :
        param_kg_loss.append(t[1])
        param_kg.append(t[0])
    elif 'dec' in t[0]:
        param_dec_loss.append(t[1])
        param_dec.append(t[0])
    else:
        param_kg_loss.append(t[1])
        param_dec_loss.append(t[1])
        param_dec.append(t[0])
        param_kg.append(t[0])
        

In [29]:
optimizer_kg_loss = optim.Adam( param_kg_loss, options.lr)

optimizer_dec_loss = optim.Adam( param_dec_loss, options.lr)


In [30]:
optimizer = optim.Adam(model_.parameters(), options.lr)

In [31]:
param = model_.parameters()

In [32]:
total_loss = 0
total_kg_loss = 0
total_dec_loss = 0

In [33]:
import math

In [ ]:

for e in range(20):
    total_loss = 0
    total_kg_loss = 0
    total_dec_loss = 0
    accuracy_ = 0 
    count_accuracy = 0
    
    for file in tqdm_notebook(training_files):
        t = pkl.load(open(file, 'rb'))
        t = padded_candidates(t)
        np.random.shuffle(t)
        n_batches = (len(t)//128)
        t = sorted(t, key = lambda x : -1 *len(x[6].split('|')))
        
        for i in tqdm_notebook(range(1, n_batches+1)):
            
            if file != testing_file:
                model_.module.dec.set_tc_ratio(1/(1+i/(n_batches)))
            else:
                model_.module.dec.set_tc_ratio(0)
                
            u1_text, u1_kb, u2_text, u2_kb, target_kb, target_utterance,\
                                cand_ent, cand_rel, cand_val, c, mem_weights\
                = mini_batch_processing(t[ (i-1) * 128 : i *128])

            u1_text = np.flip(u1_text, axis = 1).copy()
            u1_kb = np.flip(u1_kb, axis = 1).copy()

            u2_text = np.flip(u2_text, axis = 1).copy()
            u2_kb = np.flip(u2_kb, axis = 1).copy()

            u1_text = torch.LongTensor(u1_text).cuda()
            u2_text = torch.LongTensor(u2_text).cuda()

            u1_kb = torch.FloatTensor(ent_embed[u1_kb]).cuda()
            u2_kb = torch.FloatTensor(ent_embed[u2_kb]).cuda()
            system_utterance = target_utterance
            target_utterance = torch.LongTensor(target_utterance).cuda()

            cand_ent_embed  = torch.FloatTensor(ent_embed[cand_ent]).cuda()
            cand_rel_embed  = torch.FloatTensor(rel_embed[cand_rel]).cuda()
            cand_val_embed  = torch.FloatTensor(ent_embed[cand_val]).cuda()
            cand_pred_embed = torch.FloatTensor(ent_embed[cand_val]).cuda()
            mem_weights = torch.FloatTensor(mem_weights).cuda()
            
            if file != testing_file :
                #for stage in range(2):
                pred = model_(u1_text, u1_kb, u2_text, u2_kb,target_utterance,\
                                cand_ent_embed, cand_rel_embed, cand_val_embed, mem_weights )

                pred_text = pred[0]
                pred_kg = pred[1]
                
                #if torch.isnan(pred_text).sum() or torch.isnan(pred_text).sum()  > 0 :
                #break
                    
                #    if stage == 0 :
                pred_text = pred_text[:, :-1, :].contiguous().view(-1, pred_text.size(2))
                u3 = target_utterance[:, 1:].contiguous().view(-1)

                dec_loss = loss_dec(pred_text, u3)
                n_tokens = u3.ne(3).long().sum().item()
                dec_loss = dec_loss / n_tokens
                        
                #optimizer_dec_loss.zero_grad()

                #dec_loss.backward()
                #torch.nn.utils.clip_grad_norm_(param_dec_loss, 0.1)
                #optimizer_dec_loss.step()
                #else:
                c = torch.FloatTensor(c)/(0.00000001 + torch.FloatTensor(c).sum(dim = 1).unsqueeze(1)  )
                c = c.cuda()
                
                pred_kg = softmax(pred_kg) * mem_weights + 0.00000000001
                normalizer = pred_kg.sum(dim = 1).unsqueeze(1)
                pred_kg_log =  torch.log( pred_kg  / normalizer )

                kg_loss = -1 * (pred_kg_log * c).mean() 

                #optimizer_kg_loss.zero_grad()
                #kg_loss.backward()
                #torch.nn.utils.clip_grad_norm_(param_kg_loss, 0.1)
                #optimizer_kg_loss.step()

                loss = kg_loss   + dec_loss
                optimizer.zero_grad()
                loss.backward()
                torch.nn.utils.clip_grad_norm_(param, 0.1)
                optimizer.step()
                
                total_loss = total_loss + kg_loss.item() + dec_loss.item()
                total_kg_loss = total_kg_loss + kg_loss.item()
                total_dec_loss = total_dec_loss + dec_loss.item()
            else :
                pred = model_(u1_text, u1_kb, u2_text, u2_kb,target_utterance,\
                                    cand_ent_embed, cand_rel_embed, cand_val_embed, mem_weights)

                pred_text = pred[0]
                _, pos = torch.topk(pred_text,1, dim=2)
                pos.squeeze_(dim=2) 
                pos = np.array(pos.tolist())
                accuracy_ = accuracy_ + sum((system_utterance[:, 1:3 ] == pos[:,0:2]).sum(axis= 1)==2)
                count_accuracy = count_accuracy + pos.shape[0]
                
        #print("loss in epoch :", e, file, "loss :", total_loss, "loss decoder: ", \
        #              total_dec_loss, "loss kg: ", total_kg_loss)
        
        #print(q.var(dim=0).min().item(), q.var(dim=0).max().item()  )
        if file != testing_file :
            print("loss in epoch :", e, file, "loss :", total_loss, "loss decoder: ", \
                      total_dec_loss, "loss kg: ", total_kg_loss)
        else :
            print("Loss in testing", file, ": ", accuracy_*1./ count_accuracy)
        
    #np.random.shuffle(training_files)
    torch.save( model_.module.state_dict(),'ddtrain.pt')

('loss in epoch :', 0, 'dump/train_13_1.pkl', 'loss :', 785.7559123415267, 'loss decoder: ', 771.9139225482941, 'loss kg: ', 13.841989793232642)


('loss in epoch :', 0, 'dump/train_27_1.pkl', 'loss :', 1368.2612514452194, 'loss decoder: ', 1339.8430135250092, 'loss kg: ', 28.41823792021023)


('loss in epoch :', 0, 'dump/train_10_1.pkl', 'loss :', 1894.25048698904, 'loss decoder: ', 1853.4829034805298, 'loss kg: ', 40.76758350851014)


('loss in epoch :', 0, 'dump/train_26_1.pkl', 'loss :', 2406.844769820629, 'loss decoder: ', 2353.964730501175, 'loss kg: ', 52.88003931945423)


('loss in epoch :', 0, 'dump/train_3_1.pkl', 'loss :', 2899.838330135448, 'loss decoder: ', 2834.484565973282, 'loss kg: ', 65.35376416216604)


...

('loss in epoch :', 1, 'dump/valid_3_1.pkl', 'loss :', 10845.939272079035, 'loss decoder: ', 10514.155758619308, 'loss kg: ', 331.78351345972624)


('loss in epoch :', 1, 'dump/valid_2_1.pkl', 'loss :', 10919.45948273124, 'loss decoder: ', 10585.376238942146, 'loss kg: ', 334.0832437890931)


('loss in epoch :', 1, 'dump/valid_0_1.pkl', 'loss :', 11006.902993885917, 'loss decoder: ', 10669.673174023628, 'loss kg: ', 337.2298198622884)


('loss in epoch :', 1, 'dump/valid_5_1.pkl', 'loss :', 11092.82101140084, 'loss decoder: ', 10752.870062828064, 'loss kg: ', 339.9509485727758)


('Loss in testing', 'dump/test_preprocessed/test_data_file_verify.pkl', ': ', 0.0)
